In [7]:
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process.kernels import RBF
import os
import random

In [2]:
batch_size=512
epochs=300
seed=1
cuda=False and torch.cuda.is_available()
log_interval=1
r_dim=128
z_dim=128
result_path="results_np_z_y_hat_parmu_reg/"

In [18]:
def generateData(num_pt=50,length_scale=1.0,plot=False):
    #np.random.seed(2)
    x=np.random.uniform(-3,3,num_pt)[:,None]
    x.sort(axis=0)
    k=RBF(length_scale=length_scale)(x,x)
    y=np.random.multivariate_normal(np.zeros(num_pt),k)[:,None]
    N=np.random.randint(1, num_pt)
    loc=np.random.choice(num_pt,N)
    x_context=x[loc]
    y_context=y[loc]
    if plot:
        plt.plot(x_context,y_context,'ko')
        plt.plot(x,y)
    return x_context,y_context,x,y

In [5]:
class NP(nn.Module):
    def __init__(self, r_dim,z_dim):
        super(NP, self).__init__()
        self.r_dim = r_dim
        self.z_dim = z_dim
    
        self.h_1 = nn.Linear(3, 400)
        self.h_2 = nn.Linear(400, 400)
        self.h_3 = nn.Linear(400, self.r_dim)

        self.r_to_z_mean = nn.Linear(self.r_dim, self.z_dim)
        self.r_to_z_logvar = nn.Linear(self.r_dim, self.z_dim)

        self.g_1 = nn.Linear(self.z_dim + 2, 400)
        self.g_2 = nn.Linear(400, 400)
        self.g_3 = nn.Linear(400, 400)
        self.g_4 = nn.Linear(400, 400)
        self.g_y_mu = nn.Linear(400, 1)
        self.g_y_sigma = nn.Linear(400, 1)
        

    def h(self, x_y):
        x_y = F.relu(self.h_1(x_y))
        x_y = F.relu(self.h_2(x_y))
        x_y = F.relu(self.h_3(x_y))
        return x_y

    def aggregate(self, r):
        return torch.mean(r, dim=1)

    def g(self,z_sample, x_target):
        z_et_x = torch.cat([z_sample, x_target], dim=2)
        input = F.relu(self.g_1(z_et_x))
        input = F.relu(self.g_2(input))
        input = F.relu(self.g_3(input))
        input = F.relu(self.g_4(input))
        y_mu=self.g_y_mu(input)
        y_sigma=sigma = 0.1 + 0.9 * F.softplus(self.g_y_sigma(input))
        return y_mu,y_sigma
    
    
    def xy_to_z_params(self, x, y):
        
        x_y = torch.cat([x, y], dim=2)
        
        r_i = self.h(x_y)
        r = self.aggregate(r_i)
        mu = self.r_to_z_mean(r)
        logvar = self.r_to_z_logvar(r)
        sigma=0.1+0.9*torch.sigmoid(logvar)
        return mu, sigma

    def forward(self, x_context, y_context, x_all=None, y_all=None):
        
        #produire z
        z_context_mu,z_context_sigma = self.xy_to_z_params(x_context, y_context)  # (mu, logvar) of z
        q_context = Normal(z_context_mu, z_context_sigma)
        # reconstruct the whole image including the provided context points
        x_target = x_grid.expand(y_context.shape[0], -1, -1)
        
        if self.training:  # loss function will try to keep z_context close to z_all         
            z_target_mu,z_target_sigma = self.xy_to_z_params(x_context, y_context)
            q_target = Normal(z_target_mu, z_target_sigma) 
            z_sample = q_target.rsample()
            
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            
            # Get parameters of output distribution
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)

            return p_y_pred, q_target, q_context
        else:  # at test time we don't have the image so we use only the context
            z_sample = q_context.rsample()
            z_sample = z_sample.unsqueeze(1).expand(-1, 784, -1)
            # Predict target points based on context
            y_pred_mu, y_pred_sigma = self.g(z_sample,x_target)
            p_y_pred = Normal(y_pred_mu, y_pred_sigma)
            return p_y_pred,q_context,q_context

In [6]:
from torch.distributions import Normal
from torch.distributions.kl import kl_divergence

def np_loss(p_y_pred, y_target, q_target, q_context):
    
    #return logprob + KLD
    log_likelihood = p_y_pred.log_prob(y_target).mean(dim=0).sum()
    # KL has shape (batch_size, r_dim). Take mean over batch and sum over
    # r_dim (since r_dim is dimension of normal distribution)
    kl = kl_divergence(q_target, q_context).mean(dim=0).sum()
    return -log_likelihood + kl
    
    

In [7]:
model = NP(r_dim,z_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
x_grid = generate_grid(28, 28)
os.makedirs(result_path, exist_ok=True)

In [8]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (y_all, _) in enumerate(train_loader):
        
        batch_size = y_all.shape[0]
        
        y_all = y_all.to(device).view(batch_size, -1, 1)
        
        N = random.randint(1, 784)  # number of context points
        context_idx = get_context_idx(N)
        x_context = idx_to_x(context_idx, batch_size)
        y_context = idx_to_y(context_idx, y_all)
        x_all = x_grid.expand(batch_size, -1, -1)

        optimizer.zero_grad()
        
        p_y_pred, q_target, q_context = model(x_context, y_context, x_all, y_all)
        
        loss = np_loss(p_y_pred, y_all, q_target, q_context)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(y_all), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                       loss.item() / len(y_all)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
        epoch, train_loss / len(train_loader.dataset)))

In [17]:
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (y_all, _) in enumerate(test_loader):
            y_all = y_all.to(device).view(y_all.shape[0], -1, 1)
            batch_size = y_all.shape[0]

            N = 300
            context_idx = get_context_idx(N)
            x_context = idx_to_x(context_idx, batch_size)
            y_context = idx_to_y(context_idx, y_all)
            
            p_y_pred, q_target, q_context = model(x_context, y_context)
        
            
            test_loss += np_loss(p_y_pred, y_all, q_target, q_context).item()

            if i == 0:  # save PNG of reconstructed examples
                plot_Ns = [10, 100, 300, 784]
                num_examples = min(batch_size, 16)
                for N in plot_Ns:
                    recons = []
                    context_idx = get_context_idx(N)
                    x_context = idx_to_x(context_idx, batch_size)
                    y_context = idx_to_y(context_idx, y_all)
                    for d in range(5):
                        p_y_pred, _, _ = model(x_context, y_context)
                        
                        recons.append(p_y_pred.rsample()[:num_examples])     
                    recons = torch.cat(recons).view(-1, 1, 28, 28).expand(-1, 3, -1, -1)
                    background = torch.tensor([0., 0., 1.], device=device)
                    background = background.view(1, -1, 1).expand(num_examples, 3, 784).contiguous()
                    context_pixels = y_all[:num_examples].view(num_examples, 1, -1)[:, :, context_idx]
                    context_pixels = context_pixels.expand(num_examples, 3, -1)
                    background[:, :, context_idx] = context_pixels
                    comparison = torch.cat([background.view(-1, 3, 28, 28),
                                            recons])
                    save_image(comparison.cpu(),
                               result_path+"ep_" + str(epoch) +
                               "_nps_" + str(N) + ".png", nrow=num_examples)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
for epoch in range(14, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 14 [0/60000 (0%)]	Loss: -0.767608
Train Epoch: 14 [512/60000 (1%)]	Loss: -0.751735
Train Epoch: 14 [1024/60000 (2%)]	Loss: -0.757991
Train Epoch: 14 [1536/60000 (3%)]	Loss: -0.745510
Train Epoch: 14 [2048/60000 (3%)]	Loss: -0.759890
Train Epoch: 14 [2560/60000 (4%)]	Loss: -0.755115
Train Epoch: 14 [3072/60000 (5%)]	Loss: -0.758321
Train Epoch: 14 [3584/60000 (6%)]	Loss: -0.785377
Train Epoch: 14 [4096/60000 (7%)]	Loss: -0.796116
Train Epoch: 14 [4608/60000 (8%)]	Loss: -0.762706
Train Epoch: 14 [5120/60000 (8%)]	Loss: -0.769937
Train Epoch: 14 [5632/60000 (9%)]	Loss: -0.725292
Train Epoch: 14 [6144/60000 (10%)]	Loss: -0.764569
Train Epoch: 14 [6656/60000 (11%)]	Loss: -0.746527
Train Epoch: 14 [7168/60000 (12%)]	Loss: -0.748802
Train Epoch: 14 [7680/60000 (13%)]	Loss: -0.766164
Train Epoch: 14 [8192/60000 (14%)]	Loss: -0.738145
Train Epoch: 14 [8704/60000 (14%)]	Loss: -0.738172
Train Epoch: 14 [9216/60000 (15%)]	Loss: -0.753874
Train Epoch: 14 [9728/60000 (16%)]	Loss: -0.769

Train Epoch: 15 [20480/60000 (34%)]	Loss: -0.782769
Train Epoch: 15 [20992/60000 (35%)]	Loss: -0.712952
Train Epoch: 15 [21504/60000 (36%)]	Loss: -0.757199
Train Epoch: 15 [22016/60000 (36%)]	Loss: -0.750142
Train Epoch: 15 [22528/60000 (37%)]	Loss: -0.757212
Train Epoch: 15 [23040/60000 (38%)]	Loss: -0.792166
Train Epoch: 15 [23552/60000 (39%)]	Loss: -0.760809
Train Epoch: 15 [24064/60000 (40%)]	Loss: -0.764642
Train Epoch: 15 [24576/60000 (41%)]	Loss: -0.773502
Train Epoch: 15 [25088/60000 (42%)]	Loss: -0.764615
Train Epoch: 15 [25600/60000 (42%)]	Loss: -0.749009
Train Epoch: 15 [26112/60000 (43%)]	Loss: -0.785451
Train Epoch: 15 [26624/60000 (44%)]	Loss: -0.754639
Train Epoch: 15 [27136/60000 (45%)]	Loss: -0.742397
Train Epoch: 15 [27648/60000 (46%)]	Loss: -0.777659
Train Epoch: 15 [28160/60000 (47%)]	Loss: -0.771843
Train Epoch: 15 [28672/60000 (47%)]	Loss: -0.764871
Train Epoch: 15 [29184/60000 (48%)]	Loss: -0.623917
Train Epoch: 15 [29696/60000 (49%)]	Loss: -0.779492
Train Epoch:

Train Epoch: 16 [40448/60000 (67%)]	Loss: -0.773893
Train Epoch: 16 [40960/60000 (68%)]	Loss: -0.771169
Train Epoch: 16 [41472/60000 (69%)]	Loss: -0.761721
Train Epoch: 16 [41984/60000 (69%)]	Loss: -0.790034
Train Epoch: 16 [42496/60000 (70%)]	Loss: -0.763317
Train Epoch: 16 [43008/60000 (71%)]	Loss: -0.787948
Train Epoch: 16 [43520/60000 (72%)]	Loss: -0.783453
Train Epoch: 16 [44032/60000 (73%)]	Loss: -0.765876
Train Epoch: 16 [44544/60000 (74%)]	Loss: -0.776782
Train Epoch: 16 [45056/60000 (75%)]	Loss: -0.765851
Train Epoch: 16 [45568/60000 (75%)]	Loss: -0.731538
Train Epoch: 16 [46080/60000 (76%)]	Loss: -0.729736
Train Epoch: 16 [46592/60000 (77%)]	Loss: -0.776219
Train Epoch: 16 [47104/60000 (78%)]	Loss: -0.756522
Train Epoch: 16 [47616/60000 (79%)]	Loss: -0.764337
Train Epoch: 16 [48128/60000 (80%)]	Loss: -0.764561
Train Epoch: 16 [48640/60000 (81%)]	Loss: -0.756731
Train Epoch: 16 [49152/60000 (81%)]	Loss: -0.748479
Train Epoch: 16 [49664/60000 (82%)]	Loss: -0.786177
Train Epoch:

====> Test set loss: -0.7843
Train Epoch: 18 [0/60000 (0%)]	Loss: -0.752082
Train Epoch: 18 [512/60000 (1%)]	Loss: -0.729629
Train Epoch: 18 [1024/60000 (2%)]	Loss: -0.733355
Train Epoch: 18 [1536/60000 (3%)]	Loss: -0.751919
Train Epoch: 18 [2048/60000 (3%)]	Loss: -0.760740
Train Epoch: 18 [2560/60000 (4%)]	Loss: -0.753886
Train Epoch: 18 [3072/60000 (5%)]	Loss: -0.762269
Train Epoch: 18 [3584/60000 (6%)]	Loss: -0.710559
Train Epoch: 18 [4096/60000 (7%)]	Loss: -0.777625
Train Epoch: 18 [4608/60000 (8%)]	Loss: -0.728755
Train Epoch: 18 [5120/60000 (8%)]	Loss: -0.761260
Train Epoch: 18 [5632/60000 (9%)]	Loss: -0.680493
Train Epoch: 18 [6144/60000 (10%)]	Loss: -0.749966
Train Epoch: 18 [6656/60000 (11%)]	Loss: -0.791898
Train Epoch: 18 [7168/60000 (12%)]	Loss: -0.786334
Train Epoch: 18 [7680/60000 (13%)]	Loss: -0.764163
Train Epoch: 18 [8192/60000 (14%)]	Loss: -0.782414
Train Epoch: 18 [8704/60000 (14%)]	Loss: -0.756358
Train Epoch: 18 [9216/60000 (15%)]	Loss: -0.755179
Train Epoch: 18 [9

In [22]:
torch.save(model.state_dict(), result_path+"model.pt")

In [20]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
epochs=500